# BioSimSpace
Author: Rory Portman <br>
Email:&nbsp;&nbsp; portmanrk@gmail.com

_This notebook forms part of my BSc research project on Binding Free Energy Calculation protocols for Small molecule-RNA Complexes._

# Molecular Dynamics Simulations Setup and Analysis for Small Molecule-RNA Complexes

## Introduction

__This notebook covers the necessary steps to setup, run and analyse Molecular Dynamics Simulations for Small Molecule-RNA complexes. Included in this notebook:__

* Loading, parameterising, solvating and merging the Ligands, the RNA and the Waters.
* Running Equilibration and Minimisation for the Small Molecule:RNA system.
* Writing scripts to initiate 100 ns molecular dynamics simulations using various simulation engines.
* Analysing MD simulations and producing plots for reports.

__Notebooks containing follow-on steps to perform and analyse Relative Binding Free Energy Calculations can be found at:https://github.com/michellab/RNA_AFE. These notebooks contain code for:__

* Loading, parameterising, solvating and merging ligands, the RNA and the waters for RBFE calculations. Includes making merged molecules.
* Generating the RNFE input (.pert) files
* Analysing the RBFE output data


__For this project I will be using small molecule ligands and an RNA PreQ1 riboswitch reported by Connely et al.<sup>1</sup> and performing MD Simulations using AMBER, <sup>2</sup> GROMACS, <sup>3</sup> and OpenMM.<sup>4</sup>__

- `[1]` _Connelly, C.M., Numata, T., Boer, R.E. et al. Synthetic ligands for PreQ1 riboswitches provide structural and mechanistic insights into targeting RNA tertiary structure. Nat Commun 10, 1501 (2019). https://doi.org/10.1038/s41467-019-09493-3_

- `[2]` _Case DA, Cheatham TE 3rd, Darden T, Gohlke H, Luo R, Merz KM Jr, Onufriev A, Simmerling C, Wang B, Woods RJ. The Amber biomolecular simulation programs. J Comput Chem. 2005 Dec;26(16):1668-88. https://doi.org/10.1002/jcc.20290 PMID: 16200636; PMCID: PMC1989667._

- `[3]` _M. J. Abraham, T. Murtola, R. Schulz, S. Páll, J. C. Smith, B. Hess and E. Lindahl, SoftwareX,2015, 1-2, 19–25.https://doi.org/10.1016/j.softx.2015.06.001_

- `[4]` _Eastman, J. Swails, J. D. Chodera, R. T. McGibbon, Y. Zhao, K. A. Beauchamp, L.-P. Wang, A. C. Simmonett, M. P. Harrigan, C. D. Stern, R. P. Wiewiora, B. R. Brooks and V. S. Pande,PLOS Computational Biology, 2017, 13, e1005659. https://doi.org/10.1371/journal.pcbi.1005659_




## 1: Imports and Inputs 

Firstly, all necessary directories were imported:

In [ ]:
import BioSimSpace as BSS
from BioSimSpace import _Exceptions
import os
import sys
import csv
import struct
import glob
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align


Then the following small molecule ligand (see below) reported in Connely et al. was obtained from the Protein Data Bank (https://www.ebi.ac.uk/pdbe/):

- __Ligand 1:__ 2-[(dibenzo[b,d]furan-2-yl)oxy]ethan-1-amine [PDB ID: 6E1S]

This ligand, along with the PreQ1 riboswitch [PDB ID: 3Q50] and waters, is loaded below. 

__Note:__

The 6E1S structure contained the PreQ1 riboswitch backbone with two omitted residues (13 and 14). To ensure a representative simulation of the ligand bound to the PreQ1 riboswitch, a structure with the cognate metabolite bound to the PreQ1 riboswitch (PDB ID: 3Q50), which included all residues, was also obtained from Connelly et al. This complete structure was then aligned with the binding pose of Ligand 1 (6E1S) using PyMOL, resulting in an RMSD of 0.4 Å. This signified that there was considerable overlap between the metabolite and the ligand, with residue 13 positioned slightly closer to the ligand than to the metabolite. The metabolite and the incomplete backbone were subsequently removed in PyMOL leaving just Ligand 1 (PDB ID: 6E1S) and the complete PreQ1 riboswitch backbone (PDB ID: 3Q50). $SO_4^{2-}$, which was present in the structure due to the crystallization and purification processes, was deleted using PyMOL as it is not typically observed under physiological conditions. The PreQ1 backbone and Ligand 1 were then protonated using PDBFixer and Open Babel (version 3.0.0) respectively. Since no parameters for the PreQ1 riboswitch were available in the AMBER GAFF2 force field, the PreQ1 backbone and the crystal structure waters were parameterized following the methodology of Hashem et al.<sup>5</sup> using tleap prior to loading.

- `[5]` _Y. Hashem and P. Auffinger, Methods, 2009, 47, 187–197. https://doi.org/10.1016/j.ymeth.2008.09.020_

In [ ]:
#LOAD LIGAND, FREE_LIGAND, RNA AND WATER

ligand = BSS.IO.readMolecules("input_rory/lig_1_protonated.pdb")[0]
free_ligand = ligand
water=BSS.IO.readMolecules("input_rory/waters.*")
rna_bases = BSS.IO.readMolecules("input_rory/input_rory_3q50/rna.*")

print ("ligand =", ligand)
print("free_ligand =",free_ligand)
print("water =",water)
print ("rna_bases =", rna_bases)

## 2: Ligand 1-PreQ1 riboswitch System Setup

Ligand 1 was parametrised using the GAFF2 AMBER forcefield. As discussed above, the RNA and Waters were parameterised prior to loading.

In [ ]:
#PARAMATERISE LIGAND, FREE_LIGAND AND WATER

gaff_free_ligand = BSS.Parameters.gaff2(free_ligand).getMolecule()
gaff_ligand = BSS.Parameters.gaff2(ligand).getMolecule()

The parameterised inputs of Ligand 1, PreQ1 and the Waters were then combined into one system called `molecules`. It is sometimes useful to view a loaded system using `BSS.Notebook.View()`.

In [ ]:
#COMBINE RNA, WATER AND LIGAND

molecules= rna_bases + gaff_ligand + water
view=BSS.Notebook.View(molecules)
view.system()

Following the creation of the Ligand 1-PreQ1 it was solvated using `BSS.Solvent`. In this case, the `tip3p` solvating model was used and an automatically calculated solvent box sizewas determined using `.getAxisAlignedBoundingBox()`. It is important to specify the counter ion concentration. In this case 0.15 M of NaCl was specified to match the physiological concentration.

In [ ]:
#DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

molecules_sol = BSS.Solvent.tip3p(molecule= molecules, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")

This process was also carried out for the unbound (free) Ligand for subsequent use in Relative Binding Free Energy calculations.

In [ ]:
#DETERMINE BOX SIZE AND SOLVATE FREE LIGAND

free_box_min, free_box_max = free_ligand.getAxisAlignedBoundingBox()
free_box_size = [y - x for x, y in zip(free_box_min, free_box_max)]
free_padding = 15 * BSS.Units.Length.angstrom
free_box_length = max(free_box_size) + 2 * free_padding
print(free_box_length)

free_molecules_sol = BSS.Solvent.tip3p(molecule= gaff_free_ligand, box=3 * [free_box_length], ion_conc = 0.15)
search = free_molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")

## 3: Ligand 1-PreQ1 riboswitch System Energy Minimisation and Equilibration

To ensure the stability of the Ligand 1-PreQ1 riboswitch system prior to molecular dynamics simulations, energy equilibration and minimisation steps were conducted using `PMEMD.CUDA`. The following approach for minimisation and equilibration is adapted from: https://github.com/michellab/BioSimSpaceTutorials/blob/69e2a9f97b2d2f5ae21c5c5b45a2992b2879621c/03_fep/execution_model/scripts/BSSligprep.py#L225 

The Ligand 1-PreQ1 riboswitch and free ligand systems were each minimised in 250 steps using `PMEMD.CUDA` at 1 atm and gradually heated from 0 K to 300 K. The systems were then equilibrated in the NVT ensemble for 5 ps with all non-solvent heavy atoms restrained, followed by 50 ps with restraints, and 50 ps without restraints. Lastly, equilibration in the NPT ensemble was performed for 200 ps with non-solvent heavy atom restraints and for another 200 ps without restraints.

Firstly, the settings, temporary directories and inputs are defined:

In [ ]:
### Settings.
minim_steps = 250
runtime_short_nvt = 5 # ps
runtime_nvt = 50 # ps RESET to 50 after TESTING ! 
runtime_npt = 200 # ps RESET to 200 after TESTING !

### preamble. tmp_dir should at some point be derived using os.environ.get("")

tmp_dir = "temp"

amber_home = "/usr/local/amber22"
pmemd_path = amber_home + "/bin/pmemd.cuda" 

#################
lig_p_solvated = free_molecules_sol
system_solvated = molecules_sol
BSS.IO.saveMolecules(f"{tmp_dir}/free_lig_s", lig_p_solvated, ["PRM7", "RST7"])
BSS.IO.saveMolecules(f"{tmp_dir}/mol_sys_s", system_solvated, ["PRM7", "RST7"])
### Minimise and equilibrate our systems to make the ligand relax inside the pocket.

lig_p_solvated = BSS.IO.readMolecules([f"{tmp_dir}/free_lig_s.prm7", f"{tmp_dir}/free_lig_s.rst7"])
system_solvated = BSS.IO.readMolecules([f"{tmp_dir}/mol_sys_s.prm7", f"{tmp_dir}/mol_sys_s.rst7"])

print ("lig_p_solvated is =", lig_p_solvated)
print ("original is =", free_molecules_sol)

The energy equilibration and minimisation is then run for the unbound (free) ligand:

In [ ]:

def runProcess(system, protocol, pmemd=False):
        """
        Given a solvated system (BSS object) and BSS protocol, run a process workflow with either 
        Sander (CPU) or pmemd.cuda (GPU). NPT is typically done with GPU to save computing time.
        Returns the processed system.
        """
        # Create the process passing a working directory.
        if not pmemd:
            process = BSS.Process.Amber(system, protocol)
        elif pmemd:
            process = BSS.Process.Amber(system, protocol, exe=pmemd_path)

        # Start the process.
        process.start()

        # Wait for the process to exit.
        process.wait()

        # Check for errors.
        if process.isError():
            print(process.stdout())
            print(process.stderr())
            raise _Exceptions.ThirdPartyError("The process exited with an error!")

        # If it worked, try to get the system. No need to block, since it's already finished.
        system = process.getSystem()

        return system


############# first minimise/equilibrate the solvated ligand.
print("\n#### Working on solvated ligand.")

print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(lig_p_solvated, protocol)

BSS.IO.saveMolecules(f"{tmp_dir}/min_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )

equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
lig_equil_fin = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

The energy equilibration and minimisation is then run for the Ligand 1-PreQ1 system:

In [ ]:
############# repeat for ligand + protein system. Include extra restrain="backbone" step.
print("\n#### Working on solvated ligand+protein.")
print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(system_solvated, protocol)


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps while restraining all backbone atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin, 
                                #restraint="backbone"
                                )
equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                )

equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil4 = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
sys_equil_fin = runProcess(equil4, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq5_mol_sys_s", system_solvated, ["PRM7", "RST7"])

# finally, save last snapshot of both equilibrated objects.
os.system("mkdir -p prep/ligands")
os.system("mkdir -p prep/protein")

print("Saving solvated/equilibrated systems.")
print("\n Ligand:")
print(lig_equil_fin)
BSS.IO.saveMolecules(f"prep_bases/ligands/free_lig_equil_solv", lig_equil_fin, ["PRM7", "RST7"])

print("\n Ligand + protein:")
print(sys_equil_fin)
BSS.IO.saveMolecules(f"prep_bases/protein/mol_sys_equil_solv", sys_equil_fin, ["PRM7", "RST7"])
print("First 20 molecules in ligand + protein system:")
for mol in sys_equil_fin.getMolecules()[:20]:
    print(mol)
print("Done.")

## 4: Running Molecular Dynamics Simulations using AMBER, GROMACS and OpenMM

Molecular dynamics simulations of the Ligand 1 in the binding site of the PreQ1 riboswitch were initiated using the BSS.Protocol and BSS.Process functions. 100 ns MD simulations of the system  were run using AMBER, GROMACS and OpenMM. The MD simulations were initiated using the following [Python Script](https://github.com/michellab/RNA_AFE.) and queued using the following [Slurm Script](https://github.com/michellab/RNA_AFE.). An example python script is shown below:

```python
import BioSimSpace as BSS
import os

print("Loading system...")
system= BSS.IO.readMolecules("../prep/protein/mol_sys_equil_solv.*")
protocol=BSS.Protocol.Production(runtime= 100 * BSS.Units.Time.nanosecond, report_interval=10_000)
process = BSS.Process.Amber(system, protocol,exe = "/usr/local/amber22/bin/pmemd.cuda", work_dir="output_amber")
process.start()
process.wait()

## 5: Molecular Dynamics Simulation Analysis

The trajectories from the MD simulations were initially visualised using PyMOL and subsequently analysed with MDAnalysis. Firstly, the root mean square deviation (RMSD) of Ligand 1 throughout the simulations was calculated relative to the ligand’s initial position using the equation below. This calculation was performed to identify the level of deviation of Ligand 1 from its starting position during the simulations.
\begin{equation}
RMSD(X, X^{ref}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} |X_i - X_i^{ref}|^2 }
\end{equation}

Possiblie selections include:
* for the PreQ1 Backbone specify `nucleic`.
* for the Ligand specifiy `resname LIG`.
* for both `nucleic or resname LIG`.
* for specific residues (e.g residue 13) specify `resid 13`.

In [ ]:
def rmsd_plot (topology, trajectory, selection, name):
    import MDAnalysis as mda
    from MDAnalysis.analysis import rms, align
    import matplotlib.pyplot as plt
    import numpy as np

    # Load the trajectory using DCDReader
    u = mda.Universe(topology, trajectory)

    # Select backbone atoms
    backbone_1 = selection

    # Calculate RMSD
    R = rms.RMSD(u, u, select= backbone_1)
    R.run()

    # Access RMSD data
    results = R.results.rmsd.T
    rmsds= results[2]
    time= results[1]
    fig, ax = plt.subplots()
    ax.plot(time, rmsds)
    ax.set_xlabel("Time / ps")
    ax.set_ylabel("RMSD / $\\AA$")
    ax.grid(True)
    plt.savefig(name)

rmsd_plot ('openmm.parm7', 'thinned_openmm.nc', 'nucleic', 'rmsd_openmm_nucleic')

Following this, the root mean square fluctuation (RMSF) for each atom in the PreQ1 riboswitch backbone was calculated relative to its average position during the simulation using the equation below. This was done to identify the areas of flexibility within the PreQ1 riboswitch backbone and hence highlight any potential sources of error for the RBFE calculations.
\begin{equation}
\rho_i = \sqrt{\langle (X_i - \langle X_i \rangle)^2 \rangle}
\end{equation}

Possiblie selections include:
* for the PreQ1 Backbone specify `nucleic`.
* for the Ligand specifiy `resname LIG`.
* for both `nucleic or resname LIG`.
* for specific residues (e.g residue 13) specify `resid 13`.

In [ ]:
def rmsf_plot (topology_1, trajectory_1, selection_1, name_1):
     u_1 = mda.Universe(topology_1, trajectory_1)
     average = align.AverageStructure(u_1,u_1, select= selection_1, ref_frame=0).run()
     ref = average.results.universe

     aligner = align.AlignTraj(u_1, ref, select= selection_1, in_memory=True).run()
     rna = u_1.select_atoms(selection_1)
     R = rms.RMSF(rna).run()
     fig, ax = plt.subplots()
     ax.plot(R.rmsf)
     ax.set_xlabel("Relative Atom Index")
     ax.set_ylabel("RMSF / $\\AA$")
     tick_step = np.arange(0,1001, 100)
     ax.set_xticks(tick_step)
     plt.savefig(name_1)

rmsf_plot ('openmm.parm7', 'thinned_openmm.nc', 'nucleic', 'rmsf_openmm_nucleic')